In [1]:
import sys
import pathlib
import pandas as pd
import numpy as np
import boto3

ROOT = pathlib.Path().absolute().parent.as_posix()
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'asdi-hackathon-sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

In [2]:
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

#role = 'role_name_with_sagemaker_permissions' #i.e. my sagemaker role otherwise proceed inside sagemaker and will auto find
role = get_execution_role()

script_processor = ScriptProcessor(command=['python3'], 
                            image_uri=processing_repository_uri, 
                            framework_version="0.20.0", role=role, 
                            instance_type="ml.t3.large", instance_count=1)

Couldn't call 'get_role' to get Role ARN from role name joshg to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::991775080983:user/joshg, therefore it cannot be used as a SageMaker execution role

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(
    code="processing.py",
    #no inputs technically needed as script is self-sufficient in this regard
    #inputs=[ProcessingInput(source=input_data, destination="asdi-hackathon/*")],
    outputs=[ProcessingOutput(output_name="final_df", source="asdi-hackathon/final-data/final_df.csv")]
)

processing_job_description = script_processor.jobs[-1].describe()
print(processing_job_description)